reformat this:
search for any data set within a small time and space window
get info for each - 
check variable names for those key words
download that dataset

In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import urllib.request
import json 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# help(ERDDAP.get_search_url)
# swath is 6 km wide, 

In [2]:
# use an actual example track that goes near wmo_42048 (27.9396, -96.8429) on 15-Feb-2024 00:53:16
min_time = "2024-02-14T22:00:00Z"
max_time = "2024-02-15T04:00:00Z"
min_lon= -97
max_lon = -96
min_lat = 19
max_lat = 30


In [3]:
# load a list of all the ERDDAPs and their urls.
# This is saved as a JSON with the code bc I cant get the direct link to the git site to work.
f = open('erddaps.json')
data_json = pd.read_json(f)
erddap_links = data_json['url'].unique()

# # load in all possible erddap servers straight from site (not working...why??):
# url = 'https://github.com/IrishMarineInstitute/awesome-erddap/blob/df70c66a0784a384f398492d356df46dae4281d4/erddaps.json'
# response = urllib.request.urlopen(url)
# data_json = json.loads(response.read())


In [4]:
# try searching for all possibly matching stations for this time/space limitation
kw = {
    "min_lon": min_lon,
    "max_lon": max_lon,
    "min_lat": min_lat,
    "max_lat": max_lat,
    "min_time": min_time,
    "max_time": max_time,
}
# initialize empty lists
allsites = list()
allurl = []
for orgs in erddap_links:
    print('Searching ' + orgs)
    
    # start making the erddap object for this erddap url 
    e = ERDDAP(
        server=orgs, 
        protocol="tabledap", # Want table data (not a grid map of data) 
        response="csv") #in csv format for pandas

    url = e.get_search_url(search_for='', response="csv", **kw)# search for a match to our time/location
    try:
        temp = pd.read_csv(url)["Dataset ID"].unique() # try to save the data
    except:
        print("----no matching sites exist") # ignore this entry if there is no data
    else:
        print("----saving matching sites")
        allsites.extend(temp) # save the site name(s)
        for _ in range(len(temp)): allurl.append(orgs) # save a url for each site

FF = pd.DataFrame(allsites,columns=['sites'])
FF = FF['sites']#.unique()
print(FF,len(FF))
print(allurl,len(allurl))
# note: some sites have messy metadata (eg listing lat/lon everywhere) and will show up as a match even when there is no relevant data.

del e

Searching https://erddap.observations.voiceoftheocean.org/erddap/
----no matching sites exist
Searching https://erddap.ogsl.ca/erddap/
----no matching sites exist
Searching https://coastwatch.pfeg.noaa.gov/erddap/
----saving matching sites
Searching https://apdrc.soest.hawaii.edu/erddap/
----no matching sites exist
Searching https://www.ncei.noaa.gov/erddap/
----no matching sites exist
Searching https://erddap.bco-dmo.org/erddap/
----saving matching sites
Searching https://erddap.emodnet.eu/erddap/
----no matching sites exist
Searching https://erddap.emodnet-physics.eu/erddap/
----saving matching sites
Searching https://erddap.marine.ie/erddap/
----no matching sites exist
Searching https://cwcgom.aoml.noaa.gov/erddap/
----no matching sites exist
Searching https://erddap.sensors.ioos.us/erddap/
----saving matching sites
Searching https://erddap.axiomdatascience.com/erddap/
----no matching sites exist
Searching http://erddap.cencoos.org/erddap/
----no matching sites exist
Searching http:

In [7]:
for jj in range(len(allurl)):
    e = ERDDAP(
        server=allurl[jj], 
        protocol="tabledap", 
        response="csv")

    e.dataset_id = FF[jj]
    e.constraints = {
#         "longitude>=":min_lon,
#         "longitude<=":min_lon,
#         "latitude>=":min_lat,
#         "latitude<=":max_lat,
        "time>=": min_time,
        "time<=": max_time}

    url = e.get_download_url()
    print(url)
    try:
        df = e.to_pandas(parse_dates=True)
        print(df.columns)
    except: 
        print('no data')
    else:
        print(df.columns)

https://coastwatch.pfeg.noaa.gov/erddap/tabledap/scrippsGliders.csv?&time>=1707948000.0&time<=1707969600.0
no data
https://coastwatch.pfeg.noaa.gov/erddap/tabledap/erdGtsppBest.csv?&time>=1707948000.0&time<=1707969600.0
Index(['trajectory', 'org', 'type', 'platform', 'cruise', 'station_id',
       'longitude (degrees_east)', 'latitude (degrees_north)', 'time (UTC)',
       'depth (m)', 'temperature (degree_C)', 'salinity (PSU)'],
      dtype='object')
Index(['trajectory', 'org', 'type', 'platform', 'cruise', 'station_id',
       'longitude (degrees_east)', 'latitude (degrees_north)', 'time (UTC)',
       'depth (m)', 'temperature (degree_C)', 'salinity (PSU)'],
      dtype='object')
https://coastwatch.pfeg.noaa.gov/erddap/tabledap/cwwcNDBCMet.csv?&time>=1707948000.0&time<=1707969600.0
Index(['station', 'longitude (degrees_east)', 'latitude (degrees_north)',
       'time (UTC)', 'wd (degrees_true)', 'wspd (m s-1)', 'gst (m s-1)',
       'wvht (m)', 'dpd (s)', 'apd (s)', 'mwd (degrees_tr

Index(['time (UTC)', 'latitude (degrees_north)', 'longitude (degrees_east)',
       'z (m)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw (cm)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_agg',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_tests',
       'sea_surface_height_above_sea_level_geoid_mllw (m)',
       'sea_surface_height_above_sea_level_geoid_mllw_qc_agg',
       'sea_surface_height_above_sea_level_geoid_mllw_qc_tests', 'station'],
      dtype='object')
Index(['time (UTC)', 'latitude (degrees_north)', 'longitude (degrees_east)',
       'z (m)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw (cm)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_agg',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_tests',
       'sea_surface_height_above_sea_level_geoid_mllw (m)',
       'sea_surf

Index(['time (UTC)', 'latitude (degrees_north)', 'longitude (degrees_east)',
       'z (m)', 'air_pressure (millibars)', 'air_pressure_qc_agg',
       'air_pressure_qc_tests', 'air_temperature (degree_Celsius)',
       'air_temperature_qc_agg', 'air_temperature_qc_tests',
       'sea_water_temperature (degree_Celsius)',
       'sea_water_temperature_qc_agg', 'sea_water_temperature_qc_tests',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw (cm)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_agg',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_tests',
       'sea_surface_height_above_sea_level_geoid_mllw (m)',
       'sea_surface_height_above_sea_level_geoid_mllw_qc_agg',
       'sea_surface_height_above_sea_level_geoid_mllw_qc_tests',
       'wind_speed_of_gust (m.s-1)', 'wind_speed_of_gust_qc_agg',
       'wind_speed_of_gust_qc_tests', 'wind_speed (m.s-1)',
       'wind_speed_qc_agg', 'wind

Index(['time (UTC)', 'latitude (degrees_north)', 'longitude (degrees_east)',
       'z (m)', 'air_pressure (millibars)', 'air_pressure_qc_agg',
       'air_pressure_qc_tests', 'air_temperature (degree_Celsius)',
       'air_temperature_qc_agg', 'air_temperature_qc_tests',
       'sea_water_temperature (degree_Celsius)',
       'sea_water_temperature_qc_agg', 'sea_water_temperature_qc_tests',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw (cm)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_agg',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_tests',
       'sea_surface_height_above_sea_level_geoid_mllw (m)',
       'sea_surface_height_above_sea_level_geoid_mllw_qc_agg',
       'sea_surface_height_above_sea_level_geoid_mllw_qc_tests',
       'wind_speed_of_gust (m.s-1)', 'wind_speed_of_gust_qc_agg',
       'wind_speed_of_gust_qc_tests', 'wind_speed (m.s-1)',
       'wind_speed_qc_agg', 'wind

Index(['time (UTC)', 'latitude (degrees_north)', 'longitude (degrees_east)',
       'z (m)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw (cm)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_agg',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_tests',
       'sea_surface_height_above_sea_level_geoid_mllw (m)',
       'sea_surface_height_above_sea_level_geoid_mllw_qc_agg',
       'sea_surface_height_above_sea_level_geoid_mllw_qc_tests', 'station'],
      dtype='object')
Index(['time (UTC)', 'latitude (degrees_north)', 'longitude (degrees_east)',
       'z (m)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw (cm)',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_agg',
       'sea_surface_height_amplitude_due_to_geocentric_ocean_tide_geoid_mllw_qc_tests',
       'sea_surface_height_above_sea_level_geoid_mllw (m)',
       'sea_surf

...

Index(['station', 'longitude (degrees_east)', 'latitude (degrees_north)',
       'time (UTC)', 'wd (degrees_true)', 'wspd (m s-1)', 'gst (m s-1)',
       'wvht (m)', 'dpd (s)', 'apd (s)', 'mwd (degrees_true)', 'bar (hPa)',
       'atmp (degree_C)', 'wtmp (degree_C)', 'dewp (degree_C)', 'vis (km)',
       'ptdy (hPa)', 'tide (m)', 'wspu (m s-1)', 'wspv (m s-1)'],
      dtype='object')
Index(['station', 'longitude (degrees_east)', 'latitude (degrees_north)',
       'time (UTC)', 'wd (degrees_true)', 'wspd (m s-1)', 'gst (m s-1)',
       'wvht (m)', 'dpd (s)', 'apd (s)', 'mwd (degrees_true)', 'bar (hPa)',
       'atmp (degree_C)', 'wtmp (degree_C)', 'dewp (degree_C)', 'vis (km)',
       'ptdy (hPa)', 'tide (m)', 'wspu (m s-1)', 'wspv (m s-1)'],
      dtype='object')
https://upwell.pfeg.noaa.gov/erddap/tabledap/nosSosATemp.csv?&time>=1707948000.0&time<=1707969600.0
Index(['longitude (degrees_east)', 'latitude (degrees_north)', 'station_id',
       'altitude (m)', 'time (UTC)', 'sensor_id',

/opt/anaconda3/lib/python3.9/site-packages/erddapy/core/interfaces.py:33: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(data, **(pandas_kwargs or {}))


Index(['fileNumber', 'data_type', 'format_version', 'handbook_version',
       'reference_date_time (UTC)', 'date_creation (UTC)', 'date_update (UTC)',
       'platform_number', 'project_name', 'pi_name', 'cycle_number',
       'direction', 'data_center', 'dc_reference', 'data_state_indicator',
       'data_mode', 'platform_type', 'float_serial_no', 'firmware_version',
       'wmo_inst_type', 'time (UTC)', 'time_qc', 'time_location (UTC)',
       'latitude (degrees_north)', 'longitude (degrees_east)', 'position_qc',
       'positioning_system', 'profile_pres_qc', 'profile_temp_qc',
       'profile_psal_qc', 'vertical_sampling_scheme', 'config_mission_number',
       'pres (decibar)', 'pres_qc', 'pres_adjusted (decibar)',
       'pres_adjusted_qc', 'pres_adjusted_error (decibar)',
       'temp (degree_Celsius)', 'temp_qc', 'temp_adjusted (degree_Celsius)',
       'temp_adjusted_qc', 'temp_adjusted_error (degree_Celsius)',
       'psal (PSU)', 'psal_qc', 'psal_adjusted (PSU)', 'psal_adju

/opt/anaconda3/lib/python3.9/site-packages/erddapy/core/interfaces.py:33: DtypeWarning: Columns (5,7,10,12,65,67,110,112,120,125,165,166,186) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(data, **(pandas_kwargs or {}))


Index(['wmo_inst_type', 'data_type', 'date_update (UTC)', 'pi_name',
       'bbp470 (m-1)', 'bbp470_qc', 'bbp470_adjusted', 'bbp470_adjusted_qc',
       'bbp470_adjusted_error', 'bbp532 (m-1)',
       ...
       'profile_nitrate_qc', 'profile_ph_in_situ_total_qc', 'profile_pres_qc',
       'profile_psal_qc', 'profile_temp_qc', 'profile_turbidity_qc',
       'profile_up_radiance412_qc', 'profile_up_radiance443_qc',
       'profile_up_radiance490_qc', 'profile_up_radiance555_qc'],
      dtype='object', length=195)
Index(['wmo_inst_type', 'data_type', 'date_update (UTC)', 'pi_name',
       'bbp470 (m-1)', 'bbp470_qc', 'bbp470_adjusted', 'bbp470_adjusted_qc',
       'bbp470_adjusted_error', 'bbp532 (m-1)',
       ...
       'profile_nitrate_qc', 'profile_ph_in_situ_total_qc', 'profile_pres_qc',
       'profile_psal_qc', 'profile_temp_qc', 'profile_turbidity_qc',
       'profile_up_radiance412_qc', 'profile_up_radiance443_qc',
       'profile_up_radiance490_qc', 'profile_up_radiance555_qc']